# necessary figure
1. chest x-ray image
2. raw gaze points (eye_gaze & reflacx)
3. fixation as scanpath (eye_gaze & reflacx)
4. fixation as heatmap (eye_gaze & reflacx)
5. text report.
6. reflacx bounding boxes


## Optional:
1. anatomal bounding boxes

In [1]:
import os, PIL

import pandas as pd
import matplotlib.pyplot as plt

from data.paths import MIMIC_EYE_PATH
from data.datasets import ReflacxDataset
from utils.eye_tracking import (
    draw_fixations,
    draw_scanpath,
    draw_heatmap,
    draw_raw,
    get_fixations_dict_from_reflacx_eye_tracking,
    get_fixations_dict_from_eyegaze_eye_tracking,
    draw_display,
    gaussian,
)

In [2]:
cxr_meta_df = pd.read_csv(os.path.join(MIMIC_EYE_PATH, "spreadsheets", "cxr_meta.csv"))

# get the one with both reflacx and eye so we can plot both.
cxr_meta_df = cxr_meta_df[cxr_meta_df.in_reflacx & cxr_meta_df.in_eye_gaze]

In [3]:
reflacx_df = pd.read_csv(os.path.join(MIMIC_EYE_PATH, "spreadsheets","REFLACX" , "metadata.csv"))
# eye_gaze_bounding_boxes_df = pd.read_csv(os.path.join(MIMIC_EYE_PATH, "spreadsheets","EyeGaze", "bounding_boxes.csv"))
# eye_gaze_eye_gaze_df = pd.read_csv(os.path.join(MIMIC_EYE_PATH, "spreadsheets","EyeGaze", "eye_gaze.csv"))
# eye_gaze_fixations_df = pd.read_csv(os.path.join(MIMIC_EYE_PATH, "spreadsheets","EyeGaze", "fixations.csv"))


In [4]:
instance = cxr_meta_df.iloc[0]

In [13]:
dicom_id = instance['dicom_id']
subject_id = instance['subject_id']
study_id = instance['study_id']

reflacx_id = None
matched_reflacx = reflacx_df[reflacx_df.dicom_id == dicom_id]

select_idx = 0
if len(matched_reflacx) > 1:
    print(f"{len(matched_reflacx)} reflacx_ids found for this dicom image. the index 0 is then used.")

matched_reflacx.iloc[select_idx]
reflacx_id = matched_reflacx.id


In [12]:
matched_reflacx.re

,Unnamed: 0,id,split,eye_tracking_data_discarded,image,dicom_id,subject_id,image_size_x,image_size_y,Airway wall thickening,...,Support devices,Wide mediastinum,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality
769,224,P300R547859,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p11/...,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,11004477,2544,3056,NaN,...,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [5]:
# find the corresponding reflacx and eye_gaze docs
dicom_id = instance['dicom_id']
reflacx_df_match = reflacx_df[reflacx_df.dicom_id == dicom_id].iloc[0]
eye_gaze_bounding_boxes_df_match = eye_gaze_bounding_boxes_df[eye_gaze_bounding_boxes_df['dicom_id'] == dicom_id]
eye_gaze_eye_gaze_df_match = eye_gaze_eye_gaze_df[eye_gaze_eye_gaze_df['DICOM_ID'] == dicom_id]
eye_gaze_fixations_df_match = eye_gaze_fixations_df[eye_gaze_fixations_df['DICOM_ID'] == dicom_id]


In [8]:
# take the first instance from reflacx.
eye_gaze_eye_gaze_df_match

,Unnamed: 0,SESSION_ID,MEDIA_ID,DICOM_ID,CNT,Time (in secs),TIMETICK(f=10000000),FPOGX,FPOGY,FPOGS,...,BKPMIN,LPMM,LPMMV,RPMM,RPMMV,SACCADE_MAG,SACCADE_DIR,VID_FRAME,X_ORIGINAL,Y_ORIGINAL
25525,25525,21,10,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,28,0.45276,3.234112e+12,0.45765,0.38235,0.00000,...,16,3.69868,1,3.70901,1,15.92546,337.80658,0.0,1029,1168
25526,25526,21,10,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,45,0.72766,3.234114e+12,0.49661,0.39660,0.46887,...,16,3.49009,1,3.49565,1,76.36993,348.37418,0.0,1252,1212
25527,25527,21,10,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,63,1.01880,3.234117e+12,0.48787,0.51067,0.74561,...,16,3.42480,1,3.59193,1,124.33324,262.24335,0.0,1202,1560
25528,25528,21,10,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,106,1.71472,3.234124e+12,0.44189,0.57360,1.03491,...,16,3.38143,1,3.49199,1,111.41269,217.59120,0.0,938,1752
25529,25529,21,10,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,121,1.95850,3.234127e+12,0.50098,0.56024,1.73181,...,16,3.42407,0,3.67275,1,114.36666,7.24792,0.0,1277,1712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25588,25588,21,10,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,1636,26.46130,3.234372e+12,0.54962,0.39369,26.18616,...,13,3.74045,1,3.72089,1,385.11423,329.80197,0.0,1556,1203
25589,25589,21,10,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,1652,26.72046,3.234374e+12,0.54501,0.38201,26.52734,...,13,3.97601,1,3.82143,1,15.40994,125.05637,0.0,1530,1167
25590,25590,21,10,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,1678,27.14075,3.234378e+12,0.64906,0.25525,26.78467,...,13,4.40700,1,3.98545,1,242.18231,34.42173,0.0,2126,780
25591,25591,21,10,9c452d99-a2e14f15-8458865f-92178374-a9e29b76,1706,27.59412,3.234383e+12,0.62358,0.76219,27.20557,...,13,3.65817,1,3.53689,1,549.67645,264.89386,0.0,1980,2329


In [13]:
pd.read_csv()

Unnamed: 0                                                                           36880
dicom_id                                      9c452d99-a2e14f15-8458865f-92178374-a9e29b76
subject_id                                                                        11004477
study_id                                                                          54420337
PerformedProcedureStepDescription                                       CHEST (PA AND LAT)
ViewPosition                                                                            PA
Rows                                                                                  3056
Columns                                                                               2544
StudyDate                                                                         21570321
StudyTime                                                                       125119.328
ProcedureCodeSequence_CodeMeaning                                       CHEST (PA AND LAT)

In [3]:
# load the the meta file we created to get all the paths.
cxr_meta_df = pd.read_csv(os.path.join(MIMIC_EYE_PATH, "spreadsheets", "cxr_meta.csv"))

cxr_meta_df[cxr_meta_df]

inspect_dicom_id  = '9c452d99-a2e14f15-8458865f-92178374-a9e29b76'

ValueError: Boolean array expected for the condition, not int64

In [ ]:
# use dicom_id 
inspect_dicom_id  = '9c452d99-a2e14f15-8458865f-92178374-a9e29b76'
reflacx_df[reflacx_df['dicom_id' == inspect_dicom_id]]


inspect_instance = cxr_meta_df.df.iloc[idx]
inspect_instance

In [ ]:
image_path = inspect_instance["image_path"]
PIL.Image.open(image_path)